# BioExplorer - CCFv3
![](../bioexplorer_ccfv3_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3, MovieMaker
from tqdm import tqdm
import os

url = 'localhost:5000'
be = BioExplorer(url)
mm = MovieMaker(be)
core = be.core_api()
be.reset_scene()

cell_radius = 5.0

### Load Atlas

In [ ]:
population_name = 'atlas_ccfv3a_averaged'

In [ ]:
cerebellum_ids = [
       512,      1025,       519,      1033,       528,      1041,      1049,
      1056,      1064,      1073,      1091,       846,        91, 589508455,
       989,      1143,      1144,      1145,       645,       912,       920,
       928,       936,       944,     10672,     10673,     10674,     10675,
     10676,     10677,       951,     10680,     10679,     10678,     10681,
     10682,       957,     10683,     10687,     10688,     10689,     10690,
     10691,     10692,     10684,     10686,       968,       976,     10705,
     10706,     10707,     10708,     10709,     10710,     10711,     10712,
     10713,       984,     10714,     10715,     10716,     10717,     10718,
       992,     10720,     10721,     10722,     10719,     10723,     10724,
     10725,     10728,      1001,     10726,     10727,     10729,     10730,
     10731,     10733,     10732,     10734,     10737,     10736,     10735,
      1007,      1017,     10685
]

cerebellum_granular_layer_ids = [
    10675, 10678, 10708, 10711, 10684, 10720, 10723, 10690, 10726, 10705, 10735, 10687, 10681, 10729, 10672, 10732
]
cerebellum_molecular_layer_ids =  [
    10677, 10680, 10710, 10713, 10686, 10722, 10725, 10692, 10728, 10707, 10737, 10689, 10683, 10731, 10674, 10734
]

cerebellum_leaf_ids = [
    10672, 10674, 10675, 10677, 10678, 10680, 10681, 10683, 10684, 10686, 10687, 10689, 10690, 10692, 846, 10705,
    10707, 10708, 10710, 10711, 10713, 91, 989, 10720, 10722, 10723, 10725, 10726, 589508455, 10728, 10729, 10731,
    10732, 10734, 10735, 10737 
]

In [ ]:
cerrebelum_filter = str(cerebellum_ids).replace('[','').replace(']','')
cerebellum_granular_layer_filter = str(cerebellum_granular_layer_ids).replace('[','').replace(']','')
cerebellum_molecular_layer_filter = str(cerebellum_molecular_layer_ids).replace('[','').replace(']','')
cerebellum_leaf_filter = str(cerebellum_leaf_ids).replace('[','').replace(']','')

atlas_center = [7075.026185864174,3862.47588645537,5699.969406561653]

In [ ]:
atlas_assembly_name = 'Cerebellum'
be.remove_assembly(atlas_assembly_name)
atlas_assembly = be.add_assembly(atlas_assembly_name)
atlas_model = be.add_atlas(
    assembly_name=atlas_assembly_name,
    population_name=population_name,
    load_cells=True, load_meshes=False, cell_radius=cell_radius,
    # cell_sql_filter='(y>%f OR z>%f)' % (4350.0, atlas_center[2]),
    cell_sql_filter='(y>%f OR z>%f)' % (atlas_center[1], atlas_center[2]),
    region_sql_filter='guid IN (%s)' % cerebellum_leaf_filter
)

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
region_colors = dict()

def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) / 256.0 for i in range(0, lv, lv // 3))

with Session(engine) as session:
    data = session.execute('select guid, color_hex_triplet from %s.region' % population_name)
    for d in data.all():
        region_colors[int(d[0])] = hex_to_rgb(d[1])

In [ ]:
model_ids = be.get_model_ids()['ids']
for model_id in model_ids:
    material_ids = be.get_material_ids(model_id)['ids'][:-1]
    palette = list()
    opacities = list()
    shading_modes = list()
    specular_exponents = list()
    user_params = list()
    refraction_indices = list()
    glossinesses = list()
    reflection_indices = list()
    for material_id in material_ids:
        c = [1, 0, 0]
        m_id = material_id
        if m_id in region_colors:
            c = region_colors[m_id]
        alpha = 1.0
        opacities.append(1.0)
        alpha = 1.0
        shading_modes.append(be.shading_mode.ELECTRON_TRANSPARENCY)
        user_params.append(3.0)
        glossinesses.append(1.0)
        specular_exponents.append(50.0)
        reflection_indices.append(0.0)
        refraction_indices.append(0.95)
        palette.append([alpha * c[0], alpha * c[1], alpha * c[2]])
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        shading_modes=shading_modes, user_parameters=user_params,
        opacities=opacities, specular_exponents=specular_exponents,
        reflection_indices=reflection_indices,
        refraction_indices=refraction_indices, glossinesses=glossinesses,
        diffuse_colors=palette, specular_colors=palette)

In [ ]:
background_color = [0.0, 0.0, 0.0]
status = core.set_renderer(
    head_light=True,
    current='advanced', background_color=background_color,
    subsampling=4, max_accum_frames=128)
params = core.AdvancedRendererParams()
params.gi_ray_length = 1e6
params.shadow_intensity = 1.0
params.soft_shadow_strength = 0.5
params.main_exposure = 1.5
params.max_ray_depth = 1
params.epsilon_multiplier = 50.0
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

In [ ]:
mm.set_camera(
    direction=[-0.5191155653746677, 0.17713002641529518, 0.836148302353031],
    origin=[15391.238801629486, 2988.224146474133, -1927.6054642919696],
    up=[-0.11693830913040375, -0.9838094070689234, 0.13581046506221545]
)

In [ ]:
# output_folder = '/scratch/images/atlas/cerebellum'

# k = 4
# spp = 128

# image_size = [k * 960, k * 540]

# mm.create_snapshot(
#     renderer='advanced', show_progress=True,
#     path=output_folder, base_name='cerebellum_layers_%dx%d_%dspp_v1' % (image_size[0], image_size[1], spp),
#     size=image_size, samples_per_pixel=spp)


In [ ]:
keys = [
    {
        'apertureRadius': 0.0,
        'direction': [-0.5191155653740487, 0.17713002641548445, 0.836148302353376],
        'focalDistance': 1000000.0,
        'origin': [15391.238801629486, 2988.224146474133, -1927.6054642919696],
        'up': [-0.11693830913051373, -0.9838094070688865, 0.1358104650623926]
    }
    ,
    {
        'apertureRadius': 0.0,
        'direction': [-0.12377833427232945, 0.4338101678600587, 0.8924615746495728],
        'focalDistance': 1000000.0,
        'origin': [12782.796248964276, 1336.183560987327, -2003.7937903171176],
        'up': [-0.16784283904235042, -0.895571900564365, 0.41204338643125266]
    }
]

double_keys = list()
for key in keys:
    double_keys.append(key)
    double_keys.append(key)
mm.build_camera_path(double_keys, 50, 25)

In [ ]:
output_folder = '/scratch/videos/atlas/%s/cerebellum_explosion/v2' % population_name
os.makedirs(output_folder, exist_ok=True)
k = 4

nb_frames = mm.get_nb_frames()
for frame in tqdm(range(nb_frames)):
    mm.set_current_frame(frame)
    mm.create_snapshot(
        renderer='advanced',
        path=output_folder, base_name='%05d' % frame,
        size=[k * 960, k * 540],
        samples_per_pixel=64)